In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
import time

from bs4 import BeautifulSoup

import asyncio
import time
import aiohttp
from aiohttp.client import ClientSession

from sys import platform
import keyboard

#Jupyter mandatory
import nest_asyncio
nest_asyncio.apply()

In [11]:
BOLD = '\033[1m'
END = '\033[0m'

## Pedantix

In [12]:
async def win_pedantix(language = 'fr', game_number = -1):
    
    start_time = time.time()
    check_time = start_time
    
    if(language == 'fr'):
        google_url = 'https://www.google.fr/'
        request_url = 'https://cemantix.certitudes.org/pedantix/score'
        file_name = 'most_common_wikipedia_words_fr.txt'
        pedantix_url = 'https://cemantix.certitudes.org/pedantix'
        pedantix_text_box = 'pedantix-guess'
        pedantix_text_btn = 'pedantix-guess-btn'
        sucess_box_xpath = "//*[@id='pedantix-success']/p[1]/b[1]"
        success_word = 'Bravo !'
        
    elif(language == 'en'):
        google_url = 'https://www.google.us/'
        request_url = 'https://cemantle.certitudes.org/pedantle/score'
        file_name = 'most_common_wikipedia_words_en.txt'
        pedantix_url = 'https://cemantle.certitudes.org/pedantle'
        pedantix_text_box = 'pedantle-guess'
        pedantix_text_btn = 'pedantle-guess-btn'
        sucess_box_xpath = "//*[@id='pedantle-success']/p[1]/b[1]"
        success_word = 'Congrats!'
        
    else:
        raise Exception("Unknown language")
        
    # Wait the good number
    word = "witness"
    resp = requests.post(request_url, json = {"word": word,"answer": [word]})
    num = resp.json()['num']
    while(num != game_number and game_number != -1):
        print("Game number is incorrect!")
        return
    
    print(BOLD + "---- Game number %d ----" % (num) + END )
        
    # Open Pedantix
    if(platform == 'win32'):
        driver = webdriver.Chrome('./chromedriver')
    if(platform == 'darwin'):
        driver = webdriver.Chrome('/Users/antoine/Applications/chromedriver')
    driver.get(pedantix_url)

    #close for pop up
    close_box = driver.find_element("id", "dialog-close")
    driver.execute_script("arguments[0].click();", close_box); #close the box

    # Load words
    positions = ['' for i in range(5000)]

    for (N_start, N_end) in [[0,2000], [2001,4000], [4001,6000], [6001,8000], [8001,10000]]:
    
        with open(file_name,'r', encoding = 'utf-8') as word_file:
            words = word_file.read().split()[N_start:N_end]

        print("%d words loaded" %(N_end))

        # Request mostly used wikipedia words to get the clear text
        
        async def send_request(session, word):    
            async with session.post(request_url, json = {"word": word,"answer": [word]}) as resp:
                data = await resp.json()
                return data['score']

        async with aiohttp.ClientSession() as session:

            tasks = []
            for word in words:
                tasks.append(asyncio.ensure_future(send_request(session, word)))

            scores = await asyncio.gather(*tasks)
            for score in scores:
                for key, value in score.items():
                    try:
                        int(value) # check if it's an int
                    except:
                        positions[int(key)] = value

        print("\nRequest sent in %.3f seconds\n" % (time.time() - check_time))
        check_time = time.time()
        
        #Create the clear text based on request's result

        text = "Wikipedia"

        for word in positions:
            if(len(word)):
                text+= ' ' + word

        print(BOLD + "Clear text obtained:" + END, text)

        print("\nConstructed the word in %.3f seconds\n" % (time.time() - check_time))
        check_time = time.time()

        # Find title on google search

        print(BOLD + "Google results:" + END)

        answers = []

        result = '10'
        url = google_url + '/search?q=' + text + '&num=' + result

        #url = (f"{google_url}/search?q={text}&num={results}")
        requests_results = requests.get(url, cookies = {'CONSENT' : 'YES+'})
        soup_link = BeautifulSoup(requests_results.content, "html.parser")
        links = soup_link.find_all("a")

        for link in links:
            link_href = link.get('href')
            if "url?q=" in link_href and not "webcache" in link_href:
                title = link.find_all('h3')
                if len(title) > 0:
                    answer = title[0].getText() 
                    print(answer)
                    answers.append(answer)

        print("\nGoogle Search in %.3f seconds\n" % (time.time() - check_time))
        check_time = time.time()

        #return

        send_box = driver.find_element("id", pedantix_text_btn)

        print(BOLD + "Answers tested: " + END, end = '')
        for answer in answers:
            for c in ",:'()-—":
                answer = answer.replace(c, ' ')
            answer_words = answer.split(' ')
            for word in answer_words:
                if( word in ['', ' ', 'Wikipedia', 'Wikipédia', 'Wiki']):
                    continue
                print(word, end = ', ')
                text_box = driver.find_element("id", pedantix_text_box)
                time.sleep(0.1)
                text_box.send_keys(word)
                driver.execute_script("arguments[0].click();", send_box)
        
        print('\n', end = '')

        print("\nPedantix solutions tested in %.3f\n" % (time.time() - check_time))
        check_tine = time.time()

        success_box = driver.find_element("xpath", sucess_box_xpath)
        if(success_box.text == success_word):
            print("------ Found in %s seconds ------" % (time.time() - start_time))
            break
        else:
            print("------ Not Found ------\n" )
        
    print("\nPress escape to close the window... ", end ='' )
    keyboard.wait("escape")
    
    driver.close()
    print(BOLD + "Done" + END)
    
    return

In [13]:
asyncio.run(win_pedantix(language = 'fr', game_number = -1))

---- Game number 332 ----


C:\Users\Antoine\AppData\Local\Temp/ipykernel_2760/2071713832.py:41: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


2000 words loaded

Request sent in 3.736 seconds

Clear text obtained: Wikipedia to en anglais voire ou est la capitale de la province de l Avec une population de habitants en elle est la plus grande ville du Canada et la quatrième plus grande ville d Amérique du Nord Elle constitue le point d de la région du fer à cheval qui correspond à une urbaine de personnes en l ouest du lac tandis que la région du Grand à parler contient une population de habitants en est un centre international des affaires de la des arts de la culture et est considéré comme l une des villes les plus et au monde La région de située sur un vaste plateau en de rivières de et de forêts a d été par des pendant plus de ans Les Britanniques les terres de la région aux en puis la ville de York en qui devient la capitale du Haut Canada en Pendant la guerre de la ville est le théâtre de la bataille de York et de par les troupes américaines York est et en Elle est désignée capitale de la province de l en lors de la canad

KeyboardInterrupt: 